In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")
submission = pd.read_csv("/kaggle/input/titanic/gender_submission.csv")

# 랜덤 시드 고정

In [3]:
import random
import os
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


1. PassengerId : 승객 번호
2. Survived : 생존여부(1: 생존, 0 : 사망)
3. Pclass : 승선권 클래스(1 : 1st, 2 : 2nd ,3 : 3rd)
4. Name : 승객 이름
5. Sex : 승객 성별
6. Age : 승객 나이
7. SibSp : 동반한 형제자매, 배우자 수
8. Parch : 동반한 부모, 자식 수
9. Ticket : 티켓의 고유 넘버
10. Fare 티켓의 요금
11. Cabin : 객실 번호
12. Embarked : 승선한 항구명(C : Cherbourg, Q : Queenstown, S : Southampton)

In [5]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


* isnull().sum() : NULL값이 되어 있는 항목의 NULL 개수를 찾아냄

In [6]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

# NULL을 mean 혹은 median으로 채우기

In [8]:
train["Age"].fillna(train["Age"].mean(), inplace = True)
test["Age"].fillna(test["Age"].mean(), inplace = True) 

In [9]:
train["Fare"].fillna(train.groupby('Pclass')['Fare'].transform('mean'), inplace=True)
test["Fare"].fillna(test.groupby('Pclass')['Fare'].transform('mean'), inplace=True)

In [10]:
train_test_data = [train, test]

for dataset in train_test_data:
    dataset['Cabin'] = dataset['Cabin'].str[:1]
cabin_mapping = {'A':0, 'B':0.4, 'C':0.8, 'D':1.2, 'E':1.6, 'F':2, 'G':2.4, 'T': 2.8}

for dataset in train_test_data:
    dataset['Cabin'] = dataset['Cabin'].map(cabin_mapping)

In [11]:
train['Cabin'].fillna(train.groupby('Pclass')['Cabin'].transform('mean'), inplace=True)
test['Cabin'].fillna(test.groupby('Pclass')['Cabin'].transform('mean'), inplace=True)

In [12]:
for dataset in train_test_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')

In [13]:
#결측값 있는지 재확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        891 non-null    float64
 11  Embarked     891 non-null    object 
dtypes: float64(3), int64(5), object(4)
memory usage: 83.7+ KB


In [14]:
train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [15]:
test.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

# Dtype object를 float이나 int로 변경

In [16]:
from sklearn.preprocessing import LabelEncoder
classle = LabelEncoder()
train['Sex'] = classle.fit_transform(train['Sex'])
test['Sex'] = classle.transform(test['Sex'])

In [17]:
from sklearn.preprocessing import LabelEncoder
classle = LabelEncoder()
train['Embarked'] = classle.fit_transform(train['Embarked'])
test['Embarked'] = classle.transform(test['Embarked'])

# 필요 없는 값 지우기

In [18]:
train.drop(labels = ["PassengerId", "Name", "Ticket", "Embarked", "Cabin"], axis = 1, inplace = True)
test.drop(labels = ["PassengerId", "Name", "Ticket", "Embarked", "Cabin"], axis = 1, inplace = True)

# Train과 Label로 쪼개기

In [19]:
xtrain = train.drop(['Survived'], axis = 1)
ytrain = train['Survived']

# 다양한 학습 모델

* DecisionTreeClassifier

In [20]:
# from sklearn.tree import DecisionTreeClassifier
# tree = DecisionTreeClassifier(random_state = 42)
# tree.fit(xtrain, ytrain)
# predict = tree.predict(test)

* KNN

In [21]:
# from sklearn.neighbors import KNeighborsClassifier
# knn = KNeighborsClassifier(n_neighbors = 6)
# knn.fit(xtrain, ytrain)
# predict = knn.predict(test)

* RandomForestClassifier with RandomSearch

In [22]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.metrics import make_scorer, accuracy_score

# rfc = RandomForestClassifier(random_state = 42)
# parameters = {"n_estimators": [4, 5, 6, 7, 8, 9, 10, 15], 
#               "criterion": ["gini", "entropy"],
#               "max_features": ["auto", "sqrt", "log2"], 
#               "max_depth": [2, 3, 5, 10], 
#               "min_samples_split": [2, 3, 5, 10],
#               "min_samples_leaf": [1, 5, 8, 10]
#              }

# search = RandomizedSearchCV(rfc, parameters, scoring = make_scorer(accuracy_score))
# search = search.fit(xtrain, ytrain)

# search.best_estimator_

# rfc = search.best_estimator_
# rfc.fit(xtrain, ytrain)
# predict = rfc.predict(test)

In [23]:
# rfc = RandomForestClassifier(random_state = 0)
# rfc.fit(xtrain, ytrain)
# predict = rfc.predict(test)

* LogisticRegression

In [24]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(solver='liblinear')
log.fit(xtrain, ytrain)
predict = log.predict(test)

* LDA

In [25]:
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# LDA = LinearDiscriminantAnalysis(store_covariance=True)
# LDA.fit(xtrain, ytrain)
# predict = LDA.predict(test)

# 제출

In [26]:
submission['Survived'] = predict
submission.to_csv('submit.csv', index = False)